<a href="https://colab.research.google.com/github/PavanDaniele/drone-person-detection/blob/main/model_Training_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up: mount drive + import libraries

In [ ]:
# Run this Every time you start a new session
from google.colab import drive
drive.mount('/content/drive') # to mount google drive (to see/access it)

Mounted at /content/drive


# Normalization and Data Augmentation

Per i modelli leggeri ottimizzati come quelli per Jetson Nano, la normalizzazione delle immagini è quasi sempre richiesta prima di passarle al modello.

I modelli in PyTorch lavorano SOLO con tensori, NON con immagini PIL o array NumPy.
- ToTensor() converte un’immagine (PIL o NumPy) in un tensore PyTorch di tipo float32, formato [C, H, W] (canale, altezza, larghezza).

- Inoltre, scala i valori dei pixel da [0,255] a [0,1] automaticamente.

La normalizzazione (Normalize) funziona SOLO su tensori.
La funzione Normalize(mean, std) richiede input già in formato tensore (float) e applica lo shift/scala canale per canale.

Se provi a normalizzare un’immagine PIL o un NumPy array direttamente, ottieni errore o comportamenti inattesi.

Quindi: la sequenza è SEMPRE
(Opzionale) Resize

ToTensor()   →  Converte e scala [0,255] in [0,1]

Normalize()  →  Normalizza ogni canale secondo mean/std richiesto dal modello


\
 In sintesi:
ToTensor è indispensabile, non è solo per PyTorch, ma anche perché la normalizzazione funziona SOLO su tensori, non su immagini raw!

La normalizzazione NON sostituisce ToTensor: lavora sopra i dati già convertiti.

La sequenza ToTensor() + normalizzazione è quasi sempre necessaria, ma i dettagli della normalizzazione (mean, std, range pixel) possono cambiare in base al modello.
Vediamo la situazione per i tuoi modelli:
-


# Fine-Tuning Models

Analizzo il codice di Alexia per avere uno spunto:

1. test_comptage_img.py
Scopo:
Carica un modello YOLO addestrato e conta quanti oggetti della classe 0 (qui chiamati "oiseaux" = uccelli, ma tu potresti adattare a "persone") vengono rilevati in una singola immagine.

In [ ]:
from ultralytics import YOLO # Importa la libreria Ultralytics YOLO

# Chargement du modèle entraîné
model = YOLO("../runs/detect/train4/weights/best.pt") # Carica il modello YOLO addestrato dal file best.pt (specificare il percorso giusto)

# Prédiction sur une image
results = model.predict("test4.jpeg") # Esegue la predizione sull'immagine "test4.jpeg"

# Compte des oiseaux (classe 0)
bird_count = sum(1 for cls in results[0].boxes.cls if int(cls) == 0) # Conta quante bounding box appartengono alla classe 0

print(f"Nombre d'oiseaux : {bird_count}") # Stampa il numero di oggetti (classe 0) rilevati


Nota:

Puoi cambiare "classe 0" con "persona" se il tuo modello rileva persone come classe 0.

2. test_comptage_video.py
Scopo:
Carica un modello YOLO addestrato, effettua il tracking (con ByteTrack) e conta quanti oggetti della classe 0 ("oiseaux") entrano in un rettangolo centrale all'interno di un video.
Annota la video con bounding box, ID, conta corrente e totale degli oggetti unici che sono entrati nel rettangolo.

In [ ]:
from ultralytics import YOLO                # Importa YOLO da Ultralytics
import cv2                                  # Importa OpenCV per gestione video e immagini
import os                                   # Importa os (qui non usato, ma spesso per path)

# Charger le modèle
model = YOLO("/Users/alexiagaido--amoros/Desktop/UPV-test/entrainement_serveur/runs/detect/train9/weights/best.pt")
# Carica il modello YOLO addestrato (specifica percorso)

# Chemin de la vidéo
video_path = "img_video/video_test_1.mp4"   # Path della video da analizzare
output_path = "output_video.mp4"            # Path della video annotata in output

# Distance des bords pour le rectangle de contact (en pixels)
border_distance = 50                        # Margine dai bordi (pixels) per il rettangolo centrale

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)          # Apre la video
if not cap.isOpened():
    print("Erreur : Impossible d'ouvrir la vidéo")   # Se non apre la video, errore
    exit()

# Obtenir les propriétés de la vidéo
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))       # Ottiene larghezza frame
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))     # Ottiene altezza frame
fps = int(cap.get(cv2.CAP_PROP_FPS))                # Ottiene fps

# Définir les coordonnées du rectangle de contact
rect_x1 = border_distance                           # Coordinate x1 del rettangolo
rect_y1 = border_distance                           # Coordinate y1
rect_x2 = width - border_distance                   # Coordinate x2
rect_y2 = height - border_distance                  # Coordinate y2

# Configurer la sortie vidéo
fourcc = cv2.VideoWriter_fourcc(*"mp4v")            # Codec video per output
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # Oggetto per scrivere la video annotata

# Ensemble pour stocker les IDs uniques des oiseaux dans le rectangle
unique_bird_ids = set()                             # Insieme per salvare gli ID unici degli oggetti che sono passati nel rettangolo

while cap.isOpened():
    ret, frame = cap.read()                         # Leggi un frame
    if not ret:
        break

    # Effectuer l'inférence avec suivi
    results = model.track(frame, conf=0.5, tracker="bytetrack.yaml", persist=True)
    # Fa inferenza + tracking, usa ByteTrack, restituisce risultati con ID di tracking

    # Compter les oiseaux dans cette frame
    bird_count = 0
    if results[0].boxes.id is not None:             # Se ci sono ID di tracking
        for box, box_id in zip(results[0].boxes, results[0].boxes.id):   # Scorri bounding box e relativi ID
            # Vérifier si le centre de la bounding box est dans le rectangle
            x_center = (box.xyxy[0][0] + box.xyxy[0][2]) / 2            # Calcola centro x
            y_center = (box.xyxy[0][1] + box.xyxy[0][3]) / 2            # Calcola centro y
            if rect_x1 < x_center < rect_x2 and rect_y1 < y_center < rect_y2:   # Se centro box dentro rettangolo centrale
                unique_bird_ids.add(box_id.item())                      # Aggiungi ID a set (oggetti unici che sono passati)
                bird_count += 1                                         # Conta per questa frame

    # Annoter l'image avec les détections et IDs
    annotated_frame = results[0].plot()              # Disegna box e ID sul frame

    # Dessiner le rectangle de contact
    cv2.rectangle(
        annotated_frame,
        (rect_x1, rect_y1),
        (rect_x2, rect_y2),
        (255, 0, 0),  # Blu
        2,            # Spessore linea
    )

    # Afficher le nombre d'oiseaux dans cette frame et le total unique
    cv2.putText(
        annotated_frame,
        f"Oiseaux dans cette frame : {bird_count}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),  # Verde
        2,
    )
    cv2.putText(
        annotated_frame,
        f"Oiseaux uniques : {len(unique_bird_ids)}",
        (10, 60),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
    )

    # Écrire l'image annotée dans la vidéo de sortie
    out.write(annotated_frame)

    # Afficher l'image en temps réel
    cv2.imshow("YOLO Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Premere 'q' per uscire
        break

# Afficher le total des oiseaux uniques détectés
print(f"Nombre total d'oiseaux uniques détectés dans la vidéo : {len(unique_bird_ids)}")

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

Considerazioni tecniche
Classe 0: Il codice è pensato per oggetti "oiseaux" (uccelli) = classe 0. Se tu hai persone come classe 0, funziona identico.

Tracking (ByteTrack): Permette di assegnare un ID a ogni oggetto/persona che attraversa l’area, così da contarli solo una volta anche se si fermano/muovono nella scena.

Rettangolo di interesse: Conta solo gli oggetti il cui centro entra in una zona centrale, utile ad esempio per contare solo chi passa in una certa area (adattabile per ingressi, uscite, ecc).

Salvataggio video annotato: Il risultato è un video con box, ID e conteggi stampati sopra.